# Loss function visualisations


In [ ]:
%load_ext autoreload

import numpy as np
import math
import os
import os.path
import torch
import pickle

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from chamferdist import ChamferDistance
import open3d as o3d

from src.visualisation import *
from src.chamfer import *
from src.utils import *


This notebook contains additional experiments on loss functions.

Specifically it contains;

1. Visualising point-wise distances from a loss function
2. visualising results from point cloud reconstruction and point cloud completion methods

#### data loading and pre-processing

All data from sphere morphing must be loaded for analysis.


#### Visusalise correpsondences


In [ ]:
# visaulise losses

# load clouds
cld1_name = "data/24102.pcd"
cld2_name = "data/24106.pcd"
blueprint = "data/sample.ifc"
cuda = torch.device('cuda:0')

cld1 = np.array(o3d.io.read_point_cloud(cld1_name).points)
cld2 = np.array(o3d.io.read_point_cloud(cld2_name).points)

In [ ]:
# measure pairing loss
src = torch.Tensor([cld1, cld1])
tgt = torch.Tensor([cld2, cld2])

chamferDist = ChamferDistance()
nn = chamferDist(
    src, tgt, bidirectional=True, return_nn=True, k=1
)
dist = torch.sum(nn[0].dists) + torch.sum(nn[1].dists)

# compute the cyclical index (closest point of closest point). this should ideally be 0->n_points in order
perfect_idx = torch.range(0,nn[0].idx.shape[1]-1, dtype=int) # 0-> N_points
perfect_idx = perfect_idx[:,None] # add extra dimension
perfect_idx = perfect_idx.repeat(nn[0].idx.shape[0], 1, 1) # batch_size
true_idx_fwd = torch.gather(nn[0].idx, 1, nn[1].idx) # tgt[[src[match]]]
true_idx_bwd = torch.gather(nn[1].idx, 1, nn[0].idx) # tgt[[src[match]]]
pair_loss = torch.sum(perfect_idx == true_idx_fwd)

#print(nn[0].knn.shape, true_idx.shape, torch.flatten(true_idx[0]).shape)
#pairs = torch.gather(nn[1].knn, 1, true_idx)
paired_points_fwd = torch.stack([nn[0].knn[i][torch.flatten(nn[1].idx[i])] for i in range(nn[1].idx.shape[0])])
#paired_points_fwd = torch.stack([nn[1].knn[i][torch.flatten(true_idx_fwd[i])] for i in range(true_idx_fwd.shape[0])])
#paired_points_fwd = torch.stack([tgt[i][torch.flatten(true_idx_fwd[i])] for i in range(true_idx_fwd.shape[0])])
paired_points_fwd = paired_points_fwd.reshape((paired_points_fwd.shape[0], 
                                   paired_points_fwd.shape[1], 
                                   paired_points_fwd.shape[3])) 
pair_dist_fwd = torch.sum(torch.square(paired_points_fwd - tgt))


print("DS", true_idx_bwd.shape, nn[0].knn.shape, src.shape, nn[0].idx.shape)
paired_points_bwd = torch.stack([nn[1].knn[i][torch.flatten(nn[0].idx[i])] for i in range(nn[0].idx.shape[0])])
#paired_points_bwd = torch.stack([src[i][torch.flatten(true_idx_bwd[i])] for i in range(true_idx_bwd.shape[0])])
print(paired_points_bwd.shape)
#paired_points_bwd = torch.stack([nn[0].knn[i][torch.flatten(true_idx_bwd[i])] for i in range(true_idx_bwd.shape[0])])
paired_points_bwd = paired_points_bwd.reshape((paired_points_bwd.shape[0], 
                                   paired_points_bwd.shape[1], 
                                   paired_points_bwd.shape[3])) 
pair_dist_bwd = torch.sum(torch.square(paired_points_bwd - src))

print("pair", true_idx_bwd[0].flatten().shape)
print(dist, pair_loss, pair_dist_fwd, pair_dist_bwd)




In [ ]:
# visualise cyclical pairs
v = visualise_loss(cld1, cld2, blueprint, pairs=true_idx_bwd[0].flatten(), loss="pair", same_cloud=True)
v

In [ ]:
true_idx_fwd = torch.gather(nn[0].idx, 1, nn[1].idx) # tgt[[src[match]]]
true_idx_bwd = torch.gather(nn[1].idx, 1, nn[0].idx) # tgt[[src[match]]]

# manual chamfer loss
paired_points_bwd = torch.stack([tgt[i][torch.flatten(nn[0].idx[i])] for i in range(nn[0].idx.shape[0])])
pair_dist_bwd = paired_points_bwd - src
#pair_dist_bwd = torch.sum(torch.square(paired_points_bwd - x))
#paired_points_fwd = torch.stack([x[i][torch.flatten(nn[1].idx[i])] for i in range(nn[1].idx.shape[0])])
paired_points_fwd = torch.stack([src[i][torch.flatten(true_idx_bwd[i])] for i in range(true_idx_bwd.shape[0])])
pair_dist_fwd = paired_points_fwd - paired_points_bwd

pair_dist = pair_dist_bwd + pair_dist_fwd
pair_dist = torch.mul(torch.square(pair_dist), torch.square(pair_dist_bwd))
pair_dist = torch.sum(pair_dist) 
print(pair_dist, dist)

In [ ]:
# visualise EMD
src = torch.Tensor([cld1]).cuda()
tgt = torch.Tensor([cld2]).cuda()
loss, ass = calc_emd(src, tgt, 0.05, 1000)

ass = ass.detach().cpu().numpy()
unique = np.unique(ass)
print("unique", len(unique))

v = visualise_loss(cld1, cld2, blueprint, pairs=ass.flatten(), loss="pair", same_cloud=False)
v

In [ ]:
# visualise regular chamfer loss
v = visualise_loss(cld1, cld2, blueprint)


In [ ]:
# visualise reverse weighted chamfer loss
src = torch.Tensor([cld1]).cuda()
tgt = torch.Tensor([cld2]).cuda()
loss, ass = calc_reverse_weighted_cd_tensor(src, tgt, k=32, return_assignment=True)

ass = ass[0].detach().cpu().numpy()

unique = np.unique(ass)
print("unique", len(unique))

v = visualise_loss(cld1, cld2, blueprint, pairs=ass, loss="pair", same_cloud=False)
v

In [ ]:
# load subsampled clouds
cld1_name = "data/24102s.pcd"
cld2_name = "data/24103s.pcd"

cld1 = np.array(o3d.io.read_point_cloud(cld1_name).points)
cld2 = np.array(o3d.io.read_point_cloud(cld2_name).points)

In [ ]:
# visualise chamfer loss with coplanarity
target_pcd_tensor = torch.tensor([cld2], device=cuda)
src_pcd_tensor = torch.tensor([cld1], device=cuda)

vect, dists = knn_vectors(src_pcd_tensor, target_pcd_tensor, 3)
coplanarity = check_coplanarity(vect)
coplanarity = coplanarity[0].detach().cpu().numpy()
print("shapes", coplanarity.shape, dists.shape)

v = visualise_loss(cld1, cld2, blueprint, strength=coplanarity, k=3)
v

#### visualise cross section correspondence


In [ ]:
def draw_o3d_correspondences(a, b, ass):
    points = np.vstack([a,b])
    start_idx = np.arange(a.shape[0])
    end_idx = ass + a.shape[0]
    lines = np.stack([start_idx, end_idx], axis=1)
    print(points.shape, lines.shape, lines[:3])
    
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    
    return line_set

In [ ]:
cloud_path = "data/plane_slice2.pcd"
points = np.array(o3d.io.read_point_cloud(cloud_path).points)
# flatten
points[:,1] = 0
gt = o3d.geometry.PointCloud()
gt.points = o3d.utility.Vector3dVector(points)

# generate points with some random variations
source = o3d.geometry.PointCloud()

points2 = np.copy(points)
variation = (np.random.rand(points2.shape[0], points2.shape[1]) - 0.5)/10
variation[:,1] = 0
points2 += variation
source.points = o3d.utility.Vector3dVector(points2)


In [ ]:
%autoreload 2
# morph a shape into a different shape
# by optimising chamfer loss iteratively
# total points = num_points**2
def optimise_shape(src_pcd_tensor, tgt_pcd_tensor, iterations=5, learning_rate=0.01, loss_func= "chamfer", calc_emd= False):
    
    cuda = torch.device("cuda")
    
    # optimise
    optimizer = torch.optim.Adam([tgt_pcd_tensor], lr=learning_rate)
    intermediate, losses, assingments = [], [], []
    chamferDist = ChamferDistance()
    assignments = []

    for i in tqdm(range(iterations)):
        optimizer.zero_grad()
        
        if loss_func == "chamfer":
            nn = chamferDist(
                src_pcd_tensor, tgt_pcd_tensor, bidirectional=True, return_nn=True)
            loss = torch.sum(nn[1].dists) + torch.sum(nn[0].dists)
            assignment = [nn[0].idx[:,:,0].detach().cpu().numpy(), nn[1].idx[:,:,0].detach().cpu().numpy()]
        elif loss_func == "emd":
            loss, assignment = calc_emd(tgt_pcd_tensor, src_pcd_tensor, 0.05, 50)
            assignment = assignment.detach().cpu().numpy()
        elif loss_func == "uniform":
            loss, assignment = calc_uniform_chamfer_loss_tensor(src_pcd_tensor, tgt_pcd_tensor, return_assignment=True, k=4)
        elif loss_func == "infocd":
            loss, assignment = calc_cd_like_InfoV2(src_pcd_tensor, tgt_pcd_tensor, return_assignment=True)
        elif loss_func == "direct":
            loss = calc_direct(src_pcd_tensor, tgt_pcd_tensor)
        else:
            print("unspecified loss")
            
        #print("a", assignment[0].shape)
        loss.backward()
        optimizer.step()
        print("iteration", i, "loss", loss.item())
        
        intermediate.append(tgt_pcd_tensor.clone())

    # calculate final chamfer loss
    dist = chamferDist(src_pcd_tensor, tgt_pcd_tensor, bidirectional=True)
    if calc_emd:
        emd_loss, _ = calc_emd(tgt_pcd_tensor, src_pcd_tensor, 0.05, 50)
        print("final chamfer dist", dist.item(), "emd", emd_loss.item())
    else:
        print("final chamfer dist", dist.item())
    
    # save assignments for analysis
#     if measure_consistency:
#         with open("data/assignments_" + loss_func + ".pkl", "wb") as f:
#             pickle.dump(assingments, f)
            
    intermediate = torch.stack(intermediate)
    return intermediate

In [ ]:
# create tensors
cuda = torch.device("cuda")
x = torch.tensor([points], device=cuda)
y = torch.tensor([points2], device=cuda, requires_grad=True)

iterations = 50
intermediate = optimise_shape(x, y, iterations=iterations, learning_rate=0.01, loss_func="chamfer")
print(intermediate.shape)
intermediate = intermediate.reshape((iterations, x.shape[1], x.shape[2])).detach().cpu().numpy()
print(intermediate.shape)


In [ ]:
def draw_o3d_paths(a, stops):
    steps = stops.shape[0]
    print(stops.shape)
    points = np.vstack([a]+[stops[i] for i in range(steps)])
    line_sets = []
    #print(points.shape)
    for j in  range(steps):
        if j ==0:
            start_idx = np.arange(a.shape[0])
        else:
            start_idx = start_idx + a.shape[0]
        end_idx = start_idx + a.shape[0]
        lines = np.stack([start_idx, end_idx], axis=1)
        #print(points.shape, lines.shape, lines[:3])

        colors = [[1, 1-0.1*j, 0] for i in range(len(lines))]
        line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(points),
            lines=o3d.utility.Vector2iVector(lines),
        )
        line_set.colors = o3d.utility.Vector3dVector(colors)
        
        line_sets.append(line_set)
    
    return line_sets

In [ ]:
interm = o3d.geometry.PointCloud()
interm.points = o3d.utility.Vector3dVector(intermediate[0])

end = o3d.geometry.PointCloud()
end.points = o3d.utility.Vector3dVector(intermediate[-1])


source.paint_uniform_color([0., 0.706, 1])
end.paint_uniform_color([0., 0.706, 1])
interm.paint_uniform_color([0.5, 0.5, 0])
gt.paint_uniform_color([1., 0.706, 1])

#line_sets = draw_o3d_paths(points2, intermediate[1:])
#o3d.visualization.draw_geometries([gt, source, interm] + line_sets)

# start from the 2nd step onwards
line_sets = draw_o3d_paths(points2, intermediate)
o3d.visualization.draw_geometries([source, gt, end] + line_sets)
#o3d.visualization.draw_geometries([source, gt] )
#o3d.visualization.draw_geometries([end, gt] )
#o3d.visualization.draw_geometries([interm, gt, end] )


In [ ]:
# source.paint_uniform_color([0., 0.706, 1])
# gt.paint_uniform_color([1., 0.706, 1])
# gt.paint_uniform_color([1., 0.706, 1])

# line_set = draw_o3d_correspondences(points2, points, assignment)
# o3d.visualization.draw_geometries([gt, source, line_set])

### Visualise autoencoder results


In [ ]:
# visualise npy point cloud set

# chamfer
ch_savepath = "../PointSWD/logs2/reconstruction/model/modelnet40/"
ch_inp_list = np.load(os.path.join(ch_savepath, "input.npy"))
ch_rec_list = np.load(os.path.join(ch_savepath, "reconstruction.npy"))

# new
savepath = "../PointSWD/logs24/reconstruction/model/modelnet40/"
inp_list = np.load(os.path.join(savepath, "input.npy"))
rec_list = np.load(os.path.join(savepath, "reconstruction.npy"))

print(rec_list.shape) 

In [ ]:
ifc = setup_ifc_file(blueprint)
limit = 10
vis = []

for i in range(limit):
    vis.append(vis_ifc_and_cloud(ifc, [ch_inp_list[i].astype("float64"), ch_rec_list[i].astype("float64")]))

vis

In [ ]:
vis = []

for i in range(limit):
    vis.append(vis_ifc_and_cloud(ifc, [inp_list[i].astype("float64"), rec_list[i].astype("float64")]))

vis

In [ ]:
# compare losses
def evaluate_autoencoder_results(inp_list, rec_list, loss_funcs, batch_size=None):
    cuda = torch.device("cuda")
    if batch_size == None:
        batch_size = len(inp_list)
        
    # create empty dict
    all_losses = {func:[] for func in loss_funcs}
    
    # split into batches
    for i in tqdm(range(math.ceil(len(inp_list)/batch_size))):
        a = i*batch_size
        b = min(a+batch_size, len(inp_list))
        #print(a,b)
        
        # compute losses
        inp_tensor = torch.tensor(inp_list[a:b], device=cuda)
        rec_tensor = torch.tensor(rec_list[a:b], device=cuda)
        losses = calculate_3d_loss(inp_tensor, rec_tensor, loss_funcs)
        
        # sum losses
        for k, v in losses.items():
            all_losses[k].append(v)
            
    # average losses
    for k in all_losses:
        all_losses[k] = np.average(np.array(all_losses[k]))
    return all_losses

In [ ]:
%autoreload 2

loss_funcs = ["chamfer", "reverse", "emd"]

# chamfer
ch_loss = evaluate_autoencoder_results(ch_inp_list, ch_rec_list, loss_funcs, 512)

# new
loss = evaluate_autoencoder_results(inp_list, rec_list, loss_funcs, 512)

print("chamfer tuned", ch_loss)
print("new", loss)

### Evaluate completion results


#### DCD (VRC, MVP) / PointAttn (PCN)


In [ ]:
# produce density colourmaps that are normalised with their pairs

# get densities
def get_density(clouds):
    # compute nearest neighbours to calculated density
    clouds = torch.tensor(clouds, device="cuda")
    chamferDist = ChamferDistance()
    nn = chamferDist(clouds, clouds, bidirectional=False, return_nn=True, k=32)
    
    density = torch.mean(nn[0].dists[:,:,1:], dim=2)
    eps = 0.00001
    density = 1 / (density + eps)
    return density


# normalise for each example across prediction sets
def normalise_densities(density_sets):
    densities = torch.stack(density_sets)
    highs, lows = torch.max(densities, 2).values, torch.min(densities, 2).values
    highs, lows = torch.max(highs, 0).values, torch.min(lows, 0).values
    #print(densities.shape, highs.shape)
    
    #highs = torch.reshape(highs, densities.shape)
    highs = highs.unsqueeze(0).unsqueeze(-1)
    highs = highs.expand(densities.shape[0], densities.shape[1], densities.shape[2])
    lows = lows.unsqueeze(0).unsqueeze(-1)
    lows = lows.expand(densities.shape[0], densities.shape[1], densities.shape[2])
    diff = highs - lows
    densities = (densities - lows) / diff
    
    return densities[0], densities[1], densities[2], densities[3]
    

# represent density with colour and combine with point cloud
def get_coloured_clouds(clouds, density, colormap_name='plasma_r'):
    density = density.detach().cpu().numpy()
    colours = np.zeros((density.shape[0], density.shape[1], 4))
    colormap = plt.get_cmap(colormap_name)
    
    for i, cloud in enumerate(density):
        for j, pt in enumerate(cloud):
            colours[i,j] = colormap(pt)
            
#     clouds = clouds.detach().cpu().numpy()
    colours = colours[:,:,:3]
    pcds = []
    
    for i, cl in enumerate(clouds):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(cl)
        pcd.colors = o3d.utility.Vector3dVector(colours[i])
        pcds.append(pcd)
    
    return pcds, colours

In [ ]:
def get_cloud_list_vrc(path, prefix):
    limit = 20
    cloud_sets = []
    
    for i in range(limit):
        f_name = "fine"+str(i)+".pkl"
        with open(path + f_name, "rb") as f:
            pred, partial, gt = pickle.load(f)
            if prefix == "pred":
                clouds = pred
            elif prefix == "gt":
                clouds = gt
            else:
                clouds = partial
        cloud_sets.append(clouds.detach().cpu().numpy())
    cloud_sets = np.vstack(cloud_sets)
    print(cloud_sets.shape)
    
    return cloud_sets


In [ ]:
model = "pointattn"
if model == "VRC":
    uniform_path = "Density_aware_Chamfer_Distance/outputs_uniform/"
    dcd_path = "Density_aware_Chamfer_Distance/outputs_dcd/"

elif model == "pointattn":
    uniform_path = "../PointAttN/outputs/"
    dcd_path = "../PointAttN/outputs_cd/"

cld_uniform = get_cloud_list_vrc(uniform_path, "pred")
cld_dcd = get_cloud_list_vrc(dcd_path, "pred")
cld_gt = get_cloud_list_vrc(dcd_path, "gt")
cld_partial = get_cloud_list_vrc(dcd_path, "partial")

d_uniform = get_density(cld_uniform)
d_dcd = get_density(cld_dcd)
d_gt = get_density(cld_gt)
d_partial = get_density(cld_partial)

print(d_partial.shape)

# produce density colourmaps that are normalised with their pairs
d_uniform, d_dcd, d_gt, d_partial = normalise_densities([d_uniform, d_dcd, d_gt, d_partial])

v_uniform, _ = get_coloured_clouds(cld_uniform, d_uniform)
v_dcd, _ = get_coloured_clouds(cld_dcd, d_dcd)
v_gt, col = get_coloured_clouds(cld_gt, d_gt)
v_partial, _ = get_coloured_clouds(cld_partial, d_partial)
# c_dcd = get_colours(d_dcd)
# c_gt = get_colours(d_gt)
# c_partial = get_colours(d_partial)



In [ ]:
def view_side_by_side(v1, v2, v3, v4, i):
    # shift points
    cl1, cl2, cl3, cl4 = v1[i], v2[i], v3[i], v4[i]
    cl1.points = o3d.utility.Vector3dVector(np.array(cl1.points) - np.array([1,0,0]))
    cl3.points = o3d.utility.Vector3dVector(np.array(cl3.points) + np.array([1,0,0]))
    cl4.points = o3d.utility.Vector3dVector(np.array(cl4.points) + np.array([2,0,0]))
    
    o3d.visualization.draw_geometries([cl1, cl2, cl3, cl4])
    

In [ ]:
print(len(v_uniform))
cloud_list = [1,23, 34, 18, 36, 45]
for i in range(0,50):
#for i in cloud_list:
    print(i)
    view_side_by_side(v_gt, v_dcd, v_uniform, v_partial, i+1)


In [ ]:
# save a desired set of clouds
def save_cloud(clouds,  name, i, colours=None):
    directory = "mitsuba/"
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(clouds[i])
#     o3d.io.write_point_cloud(directory + name + str(i) + ".ply", pcd)
    with open(directory + name + str(i) + ".npy", "wb") as f:
        np.save(f, clouds[i])
    if colours is not None:
        with open(directory + name + str(i) + "c_.npy", "wb") as f:
            np.save(f, colours[i])
    
i = 4
# save_cloud(cld_uniform, "uniform", i)
# save_cloud(cld_dcd, "dcd", i)
# save_cloud(cld_partial, "partial", i)
# save_cloud(cld_gt, "gt", i)

for i in cloud_list:
    save_cloud(cld_gt, "gt", i+1)
    save_cloud(cld_uniform, "bal", i+1)
    save_cloud(cld_dcd, "dcd", i+1)
    save_cloud(cld_partial, "part", i+1)



In [ ]:
def view_cloud_list_vcn(path, prefix, ifc, col=0):
    limit = 2
    vis = []
    
    for i in range(limit):
        f_name = "fine"+str(i)+".pkl"
        with open(path + f_name, "rb") as f:
            pred, partial, gt = pickle.load(f)
            if prefix == "pred":
                clouds = pred
            elif prefix == "gt":
                clouds = gt
            else:
                clouds = partial
        for cl in clouds:
            cloud_list = [None, None, None]
            cloud_list[col] = cl.detach().cpu().numpy().astype(np.double)
            vis.append(vis_ifc_and_cloud(ifc, cloud_list))
    return vis

In [ ]:
uniform_path = "Density_aware_Chamfer_Distance/outputs_uniform/"
dcd_path = "Density_aware_Chamfer_Distance/outputs_dcd/"
ifc = setup_ifc_file(blueprint)

v_uniform = view_cloud_list_vcn(uniform_path, "pred", ifc, 0)
v_dcd = view_cloud_list_vcn(dcd_path, "pred", ifc, 1)
v_gt = view_cloud_list_vcn(dcd_path, "gt", ifc, 2)
v_partial = view_cloud_list_vcn(dcd_path, "partial", ifc, 2)

In [ ]:
print(len(v_uniform))
for i in range(len(v_uniform)):
    print(v_uniform[i], v_dcd[i], v_gt[i], v_partial[i])

#### InfoCD (Seedformer, PCN)


In [ ]:
%autoreload 2


# load NPYs from each. (GT, Complete, CD, InfoCD, UniformCD)
category_id = "02933112/"
cd_path = "../ICCV2023-HyperCD/pcn/train_pcn_Log_2024_02_22_14_49_07/outputs_cd/"
info_path = "../ICCV2023-HyperCD/pcn/train_pcn_Log_2024_02_22_14_49_07/outputs_info/"
uniform_path = "../ICCV2023-HyperCD/pcn/train_pcn_Log_2024_02_29_23_43_45/outputs_uniform/"

blueprint = "data/sample.ifc"
ifc = setup_ifc_file(blueprint)

def view_cloud_list_seedformer(category_id, path, prefix, ifc, col=0):
    files = os.listdir(os.path.join(path,category_id))
    files_filtered = [f for f in files if prefix in f]
    files_filtered.sort()
    #print(len(files), len(files_filtered))
    
    limit = 10
    vis = []
    count = 0
    
    for f in files_filtered:
        count +=1
        if count == limit:
            break
        cloud = np.load(os.path.join(path, category_id, f))
        cloud_list = [None, None, None]
        cloud_list[col] = cloud.astype("float64")
        vis.append(vis_ifc_and_cloud(ifc, cloud_list))
    return vis
    
    

In [ ]:
v_cd = view_cloud_list_seedformer(category_id, cd_path, "pred", ifc, 0)
v_info = view_cloud_list_seedformer(category_id, info_path, "pred", ifc, 1)
v_uniform = view_cloud_list_seedformer(category_id, uniform_path, "pred", ifc, 2)
v_complete = view_cloud_list_seedformer(category_id, cd_path, "gt", ifc, 0)


In [ ]:
for i in range(len(v_cd)):
    print(v_cd[i], v_info[i], v_uniform[i], v_complete[i])